In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# data science libraries
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1000)

# keras and tf
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

# other tf imports
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# print validation statement
print("all resources loaded")

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_csv('data/HAM10000_metadata.csv')
df.head()

NameError: name 'pd' is not defined

In [3]:
df['dx'].value_counts().to_dict()

In [4]:
df['image_id'] = df['image_id'].apply(lambda x: '{}.jpg'.format(x))
df.head()

In [5]:
# df['image_id'] = df['image_id'].apply(lambda x: '{}.jpg'.format(x))
im1 = os.listdir('../input/skin-cancer-mnist-ham10000/HAM10000_images_part_1')
im2 = os.listdir('../input/skin-cancer-mnist-ham10000/HAM10000_images_part_2')

print(im1[0:5])
print(im2[0:5])

In [ ]:
for i, id in enumerate(df['image_id']):
    if id in im1:
        df['image_id'][i] = 'HAM10000_images_part_1/' + df['image_id'][i]
    if id in im2:
        df['image_id'][i] = 'HAM10000_images_part_2/' + df['image_id'][i]

In [6]:
class_labels = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
df[class_labels] = 0

for i, dx in enumerate(df['dx']):
    for l in class_labels:
        if l == dx:
            df[l][i] = 1
        else:
            df[l][i] = 0
            
df.head(8000)

In [7]:
df.head(2500)

In [8]:
print(type(df['dx'].value_counts()))
class_labels = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
n_classes = 7

w = {}
vc = df['dx'].value_counts().items()
w_sum = 0
for lab, val in vc:
    w[str(lab)] = (val)
    w_sum += val

class_weight = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0
}

ind = 0
for i in w:
    class_weight[ind] = w_sum / (n_classes * w[i])
    ind += 1

class_weight

In [9]:
X = np.asarray(df['image_id'].tolist())
y = np.asarray((df['dx']).tolist()).astype('str')
print(X[0:5], y[0:5])

In [10]:
from tensorflow.keras.utils import to_categorical

print(type(y))
print(X[0:5], y[0:5])

class_labels = {
    'nv': 0,
    'mel': 1, 
    'bkl': 2, 
    'bcc': 3, 
    'akiec': 4, 
    'vasc': 5, 
    'df': 6
}

for i in range(len(y)):
    y[i] = class_labels[y[i]]

y = to_categorical(y, num_classes=7)
print(type(y))
print(X[0:5], y[0:5])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [12]:
print(X_train[0:5], X_test[0:5], y_train[0:5], y_test[0:5])

In [ ]:
# from PIL import Image

# X_train_im = []

# for i, xt in enumerate(X_train):
#     print(i)
#     im = Image.open('../input/skin-cancer-mnist-ham10000/' + str(xt))
#     im_arr = np.asarray(im)
#     X_train_im.append(im_arr)
#     im.close()
    
# X_train = np.asarray(X_train_im)
# X_train_im = None
# X_train[0:5]

In [ ]:
# np.save('X_train', X_train)

In [ ]:
# from PIL import Image

# X_test_im = []

# for i, xt in enumerate(X_test):
#     print(i)
#     im = Image.open('../input/skin-cancer-mnist-ham10000/' + str(xt))
#     im_arr = np.asarray(im)
#     X_test_im.append(im_arr)
#     im.close()
    
# X_test = np.asarray(X_test_im)
# X_test_im = None
# X_test[0:5]

In [ ]:
# np.save('X_test', X_test)

In [13]:
df = None
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')

In [14]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

def seq_model(out=7):
    # sequential attempt
    inp =  Input((450, 600, 3))
    #     scalar = StandardScaler()
    #     x = scalar.fit_transform(inp)
    x = BatchNormalization()(inp)
    
    
    x = Conv2D(64, kernel_size=5, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=5, activation='sigmoid', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=3, activation='sigmoid', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=3, activation='sigmoid', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = GlobalAveragePooling2D(data_format='channels_last')(x)
    #     x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(256, activation='sigmoid')(x)
    x = Dense(64, activation='sigmoid')(x)
#     x = Dropout(0.4)(x)
    out = Dense(out, activation='softmax')(x)
    
    model = Model(inp, out)

    print(model.summary())
    return model

In [15]:
model = seq_model(7)

In [16]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.metrics import CategoricalAccuracy as cca

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, decay=0.0)
# optimizer = RMSprop(learning_rate = 1, rho=0.9, momentum=0.5, epsilon=1e-07)
# sgd = SGD(lr = 0.01)

loss=cce()

# acc = [cca()]
acc = ['accuracy']
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=acc
)

In [ ]:
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=4, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.0000001
)

model.fit(
    (X_train, y_train),
#     validation_data=(X_test, y_test),
    epochs=5,
    verbose=1,
    callbacks=[lr_reduction]
#     class_weight=class_weight
)

All initial backed up code

In [ ]:
# from PIL import Image

# df['image'] = 0
# df['image'] = df['image_id'].apply(lambda x: np.asarray(Image.open('../input/skin-cancer-mnist-ham10000/' + str(x))))
# df.head()

In [ ]:
# class_labels = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

# X = np.asarray(df['image'].tolist())
# y = np.asarray(df['dx'])

In [ ]:
# print(X[0], y[0])

In [ ]:
from sklearn.utils import shuffle
df_shuffled = shuffle(df)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    horizontal_flip = True,
    vertical_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    channel_shift_range = 0,
    zoom_range = 0.2,
    rotation_range = 20,
    validation_split=0.2
)

In [ ]:
class_labels = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

# the different dataframes created with the ImageDatagenerator
train_df = datagen.flow_from_dataframe(
    df_shuffled,
    directory = '../input/skin-cancer-mnist-ham10000/',
    x_col = 'image_id',
    y_col = class_labels,
    subset='training',
    class_mode='raw',
#     classes=['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
)
val_df = datagen.flow_from_dataframe(
    df_shuffled,
    directory = '../input/skin-cancer-mnist-ham10000/',
    x_col = 'image_id',
    y_col = class_labels,
    subset='validation',
    class_mode='raw',
#     classes=['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
)

In [ ]:
# print(train_df.__len__())
# for x, y in train_df:
#     print(len(x))
#     break
# # for x, y in train_df:
# #     for i in y:
# #         print(i)
# #         break
# #     break
# # weight

In [ ]:
# # calculating weights
# weight = [0,0,0,0,0,0,0]
# ind = 0
# for x, y in train_df:
#     print(ind)
#     ind += 1
#     for i in y:
#         i = i.astype(int)
#         weight[i] += 1
# weight

In [ ]:
class CNN(tf.keras.Model):
    def __init__(self, out=7):
        # the model variables
        super().__init__()
        self.conv1 = Conv2D(64, kernel_size=3, activation='relu', input_shape=(450, 600, 3))
        self.maxpool1 = MaxPool2D(pool_size=(3,3))
        self.dropout1 = Dropout(0.1)
        self.conv2 = Conv2D(128, kernel_size=3, activation='relu')
        self.maxpool2 = MaxPool2D(pool_size=(3,3))
        self.dropout2 = Dropout(0.1)
        self.conv3 = Conv2D(256, kernel_size=3, activation='relu')
        self.maxpool3 = MaxPool2D(pool_size=(3,3))
        self.dropout3 = Dropout(0.1)
        self.conv4 = Conv2D(512, kernel_size=3, activation='relu')
        self.maxpool4 = MaxPool2D(pool_size=(3,3))
        self.dropout4 = Dropout(0.1)
        
        # for the dnn
        self.flatten = Flatten()
        self.dense1 = Dense(512, activation='relu')
        self.dense2 = Dense(128, activation='relu')
        self.dense3 = Dense(out, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.dropout2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.dropout3(x)
        x = self.conv4(x)
        x = self.maxpool4(x)
        x = self.dropout4(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [ ]:
model = CNN(7)

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

# adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-7)
# optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optimizer = RMSprop(learning_rate = 0.01, rho=0.9, momentum=0.01, epsilon=1e-07)
# sgd = SGD(lr = 0.01)

loss=CategoricalCrossentropy()
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_df,
    validation_data=val_df,
    epochs=5,
    verbose=1,
#     class_weight=class_weight
)

In [ ]:
import PIL

im = PIL.Image.open('../input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028933.jpg')
im = np.asarray(im)
im.shape
# im = im.reshape((1, 450, 600, 3))
# im
# im = tf.cast(im, tf.float32)
# model(im, training=False)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, MaxPool2D
from tensorflow.keras.models import Model

def inception_pseudo(freeze_layers=0):
    model = InceptionV3(weights='imagenet',include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.1)(x)
    x = Dense(4000, activation='relu')(x)
    x = Dense(1000, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(7, activation='softmax')(x)
    model = Model(model.input,out)
    for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model

In [ ]:
model = inception_pseudo(freeze_layers=30)

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.metrics import CategoricalAccuracy as cca

# adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-7)
# optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optimizer = RMSprop(learning_rate = 1, rho=0.9, momentum=0.5, epsilon=1e-07)
# sgd = SGD(lr = 0.01)

loss=cce()

acc = [cca()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=acc
)

In [ ]:
# print(val_df.iloc(0))

In [ ]:
model.fit(
    train_df,
    validation_data=val_df,
    epochs=5,
    verbose=1,
#     class_weight=class_weight
)

In [ ]:
class CNN_altered(tf.keras.Model):
    def __init__(self, out=7):
        # the model variables
        super().__init__()
        self.conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(None, 450, 600, 3))
        self.conv2 = Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.maxpool1 = MaxPool2D(pool_size=(2,2))
        
        self.conv3 = Conv2D(64, kernel_size=(3, 3), activation='relu')
        self.conv4 = Conv2D(64, kernel_size=(3, 3), activation='relu')
        self.maxpool2 = MaxPool2D(pool_size=(2,2))
        self.dropout1 = Dropout(0.2)
        
        self.conv5 = Conv2D(128, kernel_size=(3, 3), activation='relu')
        self.conv6 = Conv2D(128, kernel_size=(3, 3), activation='relu')
        self.maxpool3 = MaxPool2D(pool_size=(2,2))
        self.dropout2 = Dropout(0.3)
        
        # for the dnn
        self.flatten = Flatten()
        self.dense1 = Dense(512, activation='relu')
        self.dense2 = Dense(128, activation='relu')
        self.dropout3 = Dropout(0.4)
        self.dense3 = Dense(out, activation='softmax')
        

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxpool2(x)
        x = self.dropout1(x)
        
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.maxpool3(x)
        x = self.dropout2(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        return x

In [ ]:
model = CNN_altered(7)

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.metrics import CategoricalAccuracy as cca

# adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-7)
# optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optimizer = RMSprop(learning_rate = 1, rho=0.9, momentum=0.5, epsilon=1e-07)
# sgd = SGD(lr = 0.01)

loss=cce()

acc = [cca()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=acc
)

In [ ]:
model.fit(
    train_df,
    validation_data=val_df,
    epochs=5,
    verbose=1,
#     class_weight=class_weight
)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

def seq_model(out=7):
    # sequential attempt
    inp =  Input((450, 600, 3))
    #     scalar = StandardScaler()
    #     x = scalar.fit_transform(inp)
    x = BatchNormalization()(inp)
    
    
    x = Conv2D(64, kernel_size=5, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=5, activation='sigmoid', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=3, activation='sigmoid', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=3, activation='sigmoid', kernel_initializer='glorot_normal')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = Conv2D(64, kernel_size=3, activation='relu', kernel_initializer='glorot_normal')(x)
    x = GlobalAveragePooling2D(data_format='channels_last')(x)
    #     x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(256, activation='sigmoid')(x)
    x = Dense(64, activation='sigmoid')(x)
#     x = Dropout(0.4)(x)
    out = Dense(out, activation='softmax')(x)
    
    model = Model(inp, out)

    print(model.summary())
    return model

In [ ]:
model = seq_model(7)

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.metrics import CategoricalAccuracy as cca

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, decay=0.0)
# optimizer = RMSprop(learning_rate = 1, rho=0.9, momentum=0.5, epsilon=1e-07)
# sgd = SGD(lr = 0.01)

loss=cce()

# acc = [cca()]
acc = ['accuracy']
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=acc
)

In [ ]:
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=4, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.0000001
)

In [ ]:
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=4, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.0000001
)

model.fit(
    train_df,
    validation_data=val_df,
    epochs=5,
    verbose=1,
    callbacks=[lr_reduction]
#     class_weight=class_weight
)

In [ ]:
df_pix = pd.read_csv('../input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv')
df_pix.dataframeName = 'hmnist_28_28_RGB.csv'
nRow, nCol = df_pix.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df_pix.head()

In [ ]:
df_pix.columns.values

In [ ]:
df_pix['arr'] = 0
for i in range(nRow):
    df_pix.iloc[i]['arr'] = np.asarray(df_pix.iloc[i][[i for i in df_pix.columns.values if i !='label' and i != 'arr']]).reshape((28,28,3))

df_pix